<a href="https://colab.research.google.com/github/apollman/infm-456/blob/main/Final_Pres/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
compwh = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sp500_companies.csv", header= None, engine='python' )
compwh = compwh.iloc[1: , :]

c_headers=["Exchange", "Symbol", "Shortname", "Longname", "Sector", "Industry",
       "Currentprice", "Marketcap", "Ebitda", "Revenuegrowth", "City", "State",
      "Country", "Fulltimeemployees", "Longbusinesssummary", "Weight"]

compwh.columns= c_headers

In [4]:
to_drop = ['Marketcap', 'Ebitda', 'Revenuegrowth', 
           'City', 'State', 'Country', 
           'Fulltimeemployees', 'Weight', 'Currentprice', 
           'Longbusinesssummary']
compwh.drop(to_drop, inplace=True, axis=1)
compwh.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry
1,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics
2,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software—Infrastructure
3,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information
4,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information
5,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail


In [5]:
stckwh = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sp500_stocks.csv", header= None, engine='python')
stckwh = stckwh.iloc[1: , :]
s_headers = ['Date', 'Symbol', 'Adj_Close', 'Close', 'High', 'Low', 'Open',
       'Volume']
stckwh.columns = s_headers
stckwh = stckwh[stckwh.Date >= '2018-03-01']

In [6]:
merged_data = stckwh.merge(compwh,on=["Symbol"])
merged_data.head()

,Date,Symbol,Adj_Close,Close,High,Low,Open,Volume,Exchange,Shortname,Longname,Sector,Industry
0,2018-03-01,MMM,204.9241943359375,231.33999633789062,236.8300018310547,229.52999877929688,236.14999389648438,3487100.0,NYQ,3M Company,3M Company,Industrials,Conglomerates
1,2018-03-02,MMM,204.06497192382812,230.3699951171875,231.27000427246094,226.3300018310547,229.75,2912800.0,NYQ,3M Company,3M Company,Industrials,Conglomerates
2,2018-03-05,MMM,206.22637939453125,232.80999755859375,233.7100067138672,228.52999877929688,230.0,2235300.0,NYQ,3M Company,3M Company,Industrials,Conglomerates
3,2018-03-06,MMM,206.9793243408203,233.66000366210938,235.9199981689453,230.8000030517578,234.0500030517578,2091100.0,NYQ,3M Company,3M Company,Industrials,Conglomerates
4,2018-03-07,MMM,208.6712188720703,235.57000732421875,236.22000122070312,230.58999633789062,231.22000122070312,2213800.0,NYQ,3M Company,3M Company,Industrials,Conglomerates


In [7]:
merged_data["Adj_Close"] = merged_data["Adj_Close"].astype(str).astype(float)
merged_data["Volume"] = merged_data["Volume"].astype(str).astype(float)
merged_data["Close"] = merged_data["Close"].astype(str).astype(float)
merged_data["High"] = merged_data["High"].astype(str).astype(float)
merged_data["Low"] = merged_data["Low"].astype(str).astype(float)
merged_data["Open"] = merged_data["Open"].astype(str).astype(float)

In [8]:
#Changing NaN Values to O
merged_data["Adj_Close"]=merged_data["Adj_Close"].fillna(0)
#Removing NaN Values
dropvalue=merged_data[merged_data["Adj_Close"]== 0]
merged_data.drop(dropvalue.index,axis=0,inplace=True)

In [9]:
merged_data = merged_data.dropna()

In [10]:
merged_data['Date'] = merged_data['Date'].astype('datetime64[ns]')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
aapl = merged_data[merged_data.Symbol == 'AAPL']

In [12]:
aapl = aapl.set_index(['Date'])

In [13]:
df = aapl['Adj_Close'].pct_change() * 100

In [14]:
df = df.rename("Today")

In [15]:
df = df.reset_index()

In [16]:
df

,Date,Today
0,2018-03-01,NaN
1,2018-03-02,0.691436
2,2018-03-05,0.346185
3,2018-03-06,-0.084830
4,2018-03-07,-0.928306
...,...,...
978,2022-01-18,-1.889411
979,2022-01-19,-2.102478
980,2022-01-20,-1.034712
981,2022-01-21,-1.276513


In [17]:
for i in range(1,6):
  df['Lag '+str(i)] = df['Today'].shift(i)

In [18]:
df['Volume'] = aapl.Volume.shift(1).values/100_000_000

In [19]:
df = df.dropna()

In [20]:
df['Direction'] = [1 if i > 0 else 0 for i in df['Today']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
df = sm.add_constant(df)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [22]:
X = df[['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume']]

In [23]:
y = df.Direction

In [24]:
model = sm.Logit(y, X)

In [25]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.684345
         Iterations 4


In [26]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                  977
Model:                          Logit   Df Residuals:                      970
Method:                           MLE   Df Model:                            6
Date:                Wed, 27 Apr 2022   Pseudo R-squ.:                0.007525
Time:                        22:38:02   Log-Likelihood:                -668.61
converged:                       True   LL-Null:                       -673.67
Covariance Type:            nonrobust   LLR p-value:                    0.1189
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2260      0.157      1.439      0.150      -0.082       0.534
Lag 1         -0.0739      0.032     -2.293      0.022      -0.137      -0.011
Lag 2          0.0229      0.032      0.710      0.478      -0.040       0.086
Lag 3         -0.0131      0.032     -0.407      0.684      -0.076       0.050
Lag 4          0.0382      0.032      1.189      0.234      -0.025       0.101
Lag 5          0.0423      0.032      1.329      0.184      -0.020       0.105
Volume        -0.0471      0.116     -0.407      0.684      -0.274       0.180
==============================================================================
"""

In [27]:
prediction = result.predict(X)

In [28]:
prediction

6      0.530684
7      0.521698
8      0.519930
9      0.559603
10     0.577045
         ...   
978    0.541939
979    0.608314
980    0.556027
981    0.543698
982    0.553785
Length: 977, dtype: float64

In [29]:
def confusion_matrix(act, pred):
  predtrans = ['Up' if i > 0.5 else "Down" for i in pred]
  actuals = ['Up' if i > 0 else "Down" for i in act]
  confusion_matrix = pd.crosstab(pd.Series(actuals), 
                                 pd.Series(predtrans),
                                 rownames = ['Actual'], 
                                 colnames = ['Predicted'])
  return confusion_matrix


In [30]:
confusion_matrix(y, prediction)

Predicted,Down,Up
Actual,,
Down,75,372
Up,62,468


In [31]:
len(df)

977

In [32]:
(75+468)/977

0.5557830092118731

In [35]:
x_train =  df[df.Date.dt.year < 2021][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume']]
y_train = df[df.Date.dt.year < 2021]['Direction']
x_test = df[df.Date.dt.year == 2021][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Volume']]
y_test = df[df.Date.dt.year == 2021]['Direction']

In [36]:
model = sm.Logit(y_train, x_train)

In [37]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.680976
         Iterations 4


In [39]:
prediction = result.predict(x_test)

In [40]:
confusion_matrix(y_test, prediction)

Predicted,Down,Up
Actual,,
Down,13,105
Up,9,125


In [42]:
(13+125)/len(x_test)

0.5476190476190477

In [43]:
x_train1 =  df[df.Date.dt.year < 2021][['const', 'Lag 1', 'Lag 2']]
y_train1 = df[df.Date.dt.year < 2021]['Direction']
x_test1 = df[df.Date.dt.year == 2021][['const', 'Lag 1', 'Lag 2']]
y_test1 = df[df.Date.dt.year == 2021]['Direction']

In [45]:
model1 = sm.Logit(y_train1, x_train1)
result1 = model1.fit()
prediction1 = result1.predict(x_test1)
confusion_matrix(y_test1, prediction1)

Optimization terminated successfully.
         Current function value: 0.683249
         Iterations 4


Predicted,Down,Up
Actual,,
Down,10,108
Up,8,126


In [46]:
(10+126)/len(x_test1)

0.5396825396825397